# Data pre-processing. PSZMP

PSZMP dataset.

2/9,6,4, 1/31/2024

In [1]:
from datetime import datetime, timedelta, timezone
import json
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

from data_preprocess import read_and_parse_sourcedata

## Read the data

In [2]:
data_pth = Path(".")

Set to `True` when debugging. `csv` ﬁles will not be exported when `debug_no_csvexport = True`

In [3]:
debug_no_csvexport = False

## Process JSON file containing common mappings and strings

In [4]:
with open(data_pth / 'common_mappings.json') as f:
    common_mappings = json.load(f)

In [5]:
DatasetCode = common_mappings['datasetcode']
cruises = common_mappings['cruises']
stations = common_mappings['stations']
net_tow = common_mappings['net_tow']

iso8601_format = common_mappings['iso8601_format']
CRS = common_mappings['CRS']

## Pre-process data from csv for Event table

### Read the pre-processed csv file

`usecols` defines the columns that will be kept and the order in which they'll be organized

In [6]:
# From the Hood Canal dataset
# usecols = [
#     'sample_code', 'mesh_size', 'FWC_DS', 
#     'station', 'latitude', 'longitude', 
#     'date', 'time_start', 'time', 'day_night', 
#     'depth_min', 'depth_max',
#     'net_code', 'extra_sample_token'
# ]

usecols = [
    'Sample Code', 
    'Station', 'Latitude', 'Longitude', 'Site Name', 'Basin', 'Sampling Group',
    'Sample Date', 'Sample Time', 'Day_Night', 'time',
    'Min Tow Depth (m)', 'Max Tow Depth (m)', 'Station Depth (m)',
    'Mesh Size', 'Tow Type', 
]

# eventsource_df = read_and_parse_sourcedata(test_n_rows=1000)[usecols]
eventsource_df = read_and_parse_sourcedata()[usecols]

# TODO: Rename more columns, if needed
eventsource_df.rename(
    columns={
        'Sample Code':'sample_code',
        'Station':'station',
        'Latitude':'latitude',
        'Longitude':'longitude',
        'Min Tow Depth (m)':'depth_min', 
        'Max Tow Depth (m)':'depth_max', 
        'Mesh Size': 'mesh_size',
    },
    inplace=True
)

In [7]:
len(eventsource_df)

153825

In [8]:
eventsource_df.head()

,sample_code,station,latitude,longitude,Site Name,Basin,Sampling Group,Sample Date,Sample Time,Day_Night,time,depth_min,depth_max,Station Depth (m),mesh_size,Tow Type
0,010218ELIV1151,ELIV,48.63795,-122.5694,Eliza Island,Bellingham Bay,LUM,2018-01-02 00:00:00,11:51:00,D,2018-01-02 11:51:00-07:00,0.0,110.0,120.7,200,Vertical
1,010218ELIV1151,ELIV,48.63795,-122.5694,Eliza Island,Bellingham Bay,LUM,2018-01-02 00:00:00,11:51:00,D,2018-01-02 11:51:00-07:00,0.0,110.0,120.7,200,Vertical
2,010218ELIV1151,ELIV,48.63795,-122.5694,Eliza Island,Bellingham Bay,LUM,2018-01-02 00:00:00,11:51:00,D,2018-01-02 11:51:00-07:00,0.0,110.0,120.7,200,Vertical
3,010218ELIV1151,ELIV,48.63795,-122.5694,Eliza Island,Bellingham Bay,LUM,2018-01-02 00:00:00,11:51:00,D,2018-01-02 11:51:00-07:00,0.0,110.0,120.7,200,Vertical
4,010218ELIV1151,ELIV,48.63795,-122.5694,Eliza Island,Bellingham Bay,LUM,2018-01-02 00:00:00,11:51:00,D,2018-01-02 11:51:00-07:00,0.0,110.0,120.7,200,Vertical


### Remove duplicates

Will return only unique samples, where one row = one sample.

In [9]:
eventsource_df = eventsource_df.drop_duplicates().sort_values(by='sample_code').reset_index(drop=True)

In [10]:
len(eventsource_df)

3567

- There are 51 unique stations
- Adding 'Tow Type' didn't lead to duplication of stations. ie, each station has only one tow type.
- Adding 'Station Depth (m)' did lead to huge duplication
- Darn, adding 'Sampling Group' led to some duplication (60 records instead of 51). ie, some stations were visited by more than one 'Sampling Group'

In [11]:
stations_cols = ['station', 'latitude', 'longitude', 'Site Name', 'Basin', 'Sampling Group', 'Tow Type']

In [12]:
stations_unique_df = eventsource_df[stations_cols].drop_duplicates().sort_values(by='station').reset_index(drop=True)

In [13]:
len(stations_unique_df)

60

In [18]:
sorted(stations_unique_df['Sampling Group'].unique())

['HCSEG/DOE',
 'KC',
 'KWT',
 'LUM',
 'NIT',
 'NOAA',
 'NOAA/STIL',
 'P Stations',
 'PGST/WDFW',
 'TUL']

In [15]:
stations_unique_df['Sampling Group'].value_counts()

NIT           10
PGST/WDFW      8
TUL            8
KWT            8
NOAA           6
KC             6
NOAA/STIL      5
LUM            4
P Stations     3
HCSEG/DOE      2
Name: Sampling Group, dtype: int64

In [16]:
stations_unique_df.head(60)

,station,latitude,longitude,Site Name,Basin,Sampling Group,Tow Type
0,ADID,48.00274,-122.6374,Admiralty Inlet,Admiralty Inlet,PGST/WDFW,Oblique
1,ADIM,47.99915,-122.6664,Admiralty Inlet,Admiralty Inlet,PGST/WDFW,Oblique
2,ADIS,47.99063,-122.6872,Admiralty Inlet,Admiralty Inlet,PGST/WDFW,Oblique
3,ADIV,48.00273,-122.6360,Admiralty Inlet,Admiralty Inlet,PGST/WDFW,Vertical
4,CAMD,48.06646,-122.3956,Camano Head,Whidbey Basin,TUL,Oblique
5,CAMM,48.06536,-122.3905,Camano Head,Whidbey Basin,TUL,Oblique
6,CAMS,48.06469,-122.3886,Camano Head,Whidbey Basin,TUL,Oblique
7,CAMV,48.05901,-122.3873,Camano Head,Whidbey Basin,TUL,Vertical
8,COW1S,48.68448,-123.0400,Cowlitz,San Juan,KWT,Oblique
9,COW2D,48.68303,-123.0412,Cowlitz,San Juan,KWT,Oblique


In [7]:
# lhs_df = pd.DataFrame(source_df['lhs'].value_counts()).sort_index()

## Fix time issues

- Assign correct timezone (PDT, not UTC)
- Fill in missing `time_start`, by `sample_code`

In [ ]:
# source_df.loc[source_df.sample_code == "20130903UNDm1_335"].head(3)

Create (replace) the `time` column based on `date`, `time_start`, my custom `pdt` timezone, and `strftime`

In [ ]:
source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46276 entries, 0 to 46275
Data columns (total 42 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   BugSampleID                        46276 non-null  int64         
 1   Project                            46276 non-null  object        
 2   Sample Code                        46276 non-null  object        
 3   Sampling Group                     46276 non-null  object        
 4   Station                            46276 non-null  object        
 5   Site                               46276 non-null  object        
 6   Site Name                          46276 non-null  object        
 7   Basin                              46276 non-null  object        
 8   Sub Basin                          46276 non-null  object        
 9   Latitude                           46276 non-null  float64       
 10  Longitude                         

In [ ]:
source_df[['BugSampleID', 'Sample Code', 'Sample Date', 'Sample Time']].head()

,BugSampleID,Sample Code,Sample Date,Sample Time
0,181590,010322KSBP01D0815,2022-01-03 00:00:00,08:15:00
1,181593,010322KSBP01D0815,2022-01-03 00:00:00,08:15:00
2,181594,010322KSBP01D0815,2022-01-03 00:00:00,08:15:00
3,181604,010422LSNT01D1323,2022-01-04 00:00:00,13:23:00
4,181609,010422LSNT01D1323,2022-01-04 00:00:00,13:23:00


- **TODO:** Enable a timezone generator so the correct UTC offset (-7 or -8) is applied. Currently, it's hardwired to a constant value
- `Sample Date` includes a string for time ("00:00:00") that must be removed first.
- Will parse for seconds, but they're all 0; the resolution is minutes

In [ ]:
pdt = timezone(timedelta(hours=-7), "PDT")
source_df["time"] = pd.to_datetime(
    source_df["Sample Date"].str.split(' ').str[0] + source_df["Sample Time"], 
    format="%Y-%m-%d%H:%M:%S"
).dt.tz_localize(pdt)

No errors with datetime conversion!

In [ ]:
source_df.head()

,BugSampleID,Project,Sample Code,Sampling Group,Station,Site,Site Name,Basin,Sub Basin,Latitude,...,Genus species,Life History Stage,ID Code,Density (#/m3),Final Carbon (mg/m3),Species Size Category Aggregation,Size Category Aggregation,Prey Field Index,Collection Confidence,time
0,181590,SSMSP,010322KSBP01D0815,KC,KSBP01D,KSBP01,Point Jefferson,Central Basin,N Central Basin,47.74396,...,CYPHOCARIS CHALLENGERI,Unknown,6169341101C,0.017520,0.001591,Amphipod,Amphipod,Other,High,2022-01-03 08:15:00-07:00
1,181593,SSMSP,010322KSBP01D0815,KC,KSBP01D,KSBP01,Point Jefferson,Central Basin,N Central Basin,47.74396,...,HYPERIIDEA,Unknown,6170C,80.940469,1.606125,Amphipod,Amphipod,Hyperiid,High,2022-01-03 08:15:00-07:00
2,181594,SSMSP,010322KSBP01D0815,KC,KSBP01D,KSBP01,Point Jefferson,Central Basin,N Central Basin,47.74396,...,THEMISTO PACIFICA,Unknown,6170011003C,8.409399,0.715335,Amphipod,Amphipod,Hyperiid,High,2022-01-03 08:15:00-07:00
3,181604,SSMSP,010422LSNT01D1323,KC,LSNT01D,LSNT01,Point Williams,Central Basin,N Central Basin,47.53333,...,CYPHOCARIS CHALLENGERI,Unknown,6169341101C,0.068919,0.014498,Amphipod,Amphipod,Other,high,2022-01-04 13:23:00-07:00
4,181609,SSMSP,010422LSNT01D1323,KC,LSNT01D,LSNT01,Point Williams,Central Basin,N Central Basin,47.53333,...,THEMISTO PACIFICA,Unknown,6170011003C,14.473009,0.166785,Amphipod,Amphipod,Hyperiid,high,2022-01-04 13:23:00-07:00


## Replace incorrect `life_history_stage` values

- Simple replacement of "F1_0;_Furcilia_1_0_legs" entry with "F10;_Furcilia_10". Use pandas `replace` on the column
- EUPHAUSIA_PACIFICA and THYSANOESSA in `species` column: replace `life_history_stage` based on combined `species` and `life_history_stage` entries
  > Calyptopis 1-3 life_history_stage codes. For Euphasia Pacifica, calyptopis life stages are typically coded in life_history_stage as "Cal1;_Calyptopis_1", "Cal2;_Calyptopis_2" and "Cal3;_Calyptopis_3" (same for thysanoessa). But there are a few Euphasia records that include the following codes: "1;_CI", "2;_CII", "3;_CIII". These are the same life_history_stage codes used for copepods, copepodites C1 - C3. My guess is that they're miscoded and should be calyptopis 1-3. Can you confirm?

  Yes, these are miscoded ("1, CI" should be "Cal1, calyptopis 1," and so on).

In [ ]:
source_df["life_history_stage"].replace({"F1_0;_Furcilia_1_0_legs": "F10;_Furcilia_10"}, inplace=True)

In [ ]:
sel_species = source_df["species"].isin(["EUPHAUSIA_PACIFICA", "THYSANOESSA"])

krill_bad_life_history_stages = {
    "1;_CI": "Cal1;_Calyptopis_1", 
    "2;_CII": "Cal2;_Calyptopis_2",
    "3;_CIII": "Cal3;_Calyptopis_3",
}

In [ ]:
for old_lhs,new_lhs in krill_bad_life_history_stages.items():
    source_df.loc[sel_species & source_df["life_history_stage"].str.startswith(old_lhs), "life_history_stage"] = new_lhs

### Parse `life_history_stage`

#### Parse into `lhs_0` and `lhs_1` tokens

In [ ]:
source_df[['lhs_0', 'lhs_1']] = pd.DataFrame(
    source_df['life_history_stage'].str.split(';_').to_list(), 
    index=source_df.index
)

In [ ]:
source_df[['life_history_stage', 'lhs_0', 'lhs_1']].value_counts(dropna=False, sort=False).sort_index(ascending=True)

life_history_stage  lhs_0             lhs_1       
1;_CI               1                 CI               120
2;_CII              2                 CII              176
3;_CIII             3                 CIII             272
4;_CIV              4                 CIV              492
5;_CV               5                 CV               611
Adult               Adult             NaN              194
Bract               Bract             NaN                6
Cal1;_Calyptopis_1  Cal1              Calyptopis_1      49
Cal2;_Calyptopis_2  Cal2              Calyptopis_2      34
Cal3;_Calyptopis_3  Cal3              Calyptopis_3      42
Copepodite          Copepodite        NaN              265
Cyphonaut           Cyphonaut         NaN               81
Cyprid_larva        Cyprid_larva      NaN               36
Egg                 Egg               NaN               91
Eudoxid             Eudoxid           NaN                5
F10;_Furcilia_10    F10               Furcilia_10        3
F1;_F

### Examine original issues with incorrect entries

**NOTE:** The statements below will show the original issues only when run with the `lhs_0` and `lhs_1` columns parsed from the *original, unaltered* `life_history_stage` entries. I've commented them out for that reason.

Examine use of CI-V life stages (Copepodites?) with EUPHAUSIA_PACIFICA krill. Note that only CI-III are used with krill, and only a small subset of the totals (9, 9 & 6, respectively)

In [ ]:
# source_df[source_df.species == 'EUPHAUSIA_PACIFICA'][['lhs_0', 'lhs_1']].value_counts(dropna=False).sort_index(ascending=True)

Confirm that 'Cal1', 'Cal2', 'Cal3' are only used with krill

In [ ]:
# pd.DataFrame(
#     source_df[source_df.lhs_0.isin(['Cal1', 'Cal2', 'Cal3'])]
#     [['lhs_0', 'lhs_1', 'species']]
#     .value_counts(dropna=False)
#     .sort_index(ascending=True)
# ).head(60)

In [ ]:
# pd.DataFrame(
#     source_df.lhs_0.isin(['1', '2', '3', '4', '5'])
#     [['lhs_0', 'lhs_1', 'species']]
#     .value_counts(dropna=False)
#     .sort_index(ascending=True)
# ).head(60)

## Move the taxonomy replacements here??

No, b/c the taxonomy notebook also does a lot of other taxonomy querying and mappings, so it's best to keep it all there.

## Package versions

In [ ]:
print(
    f"{datetime.utcnow()} +00:00\n"
    f"pandas: {pd.__version__}, geopandas: {gpd.__version__}"
)

2023-11-01 02:47:27.815453 +00:00
pandas: 1.5.3, geopandas: 0.12.2
